In [1]:
!pip install dgl-cu111 dglgo -f https://data.dgl.ai/wheels/repo.html
# !pip install dgl dglgo -f https://data.dgl.ai/wheels/repo.html

In [2]:
import dgl.data
dataset = dgl.data.CoraGraphDataset()
# dataset = dgl.data.CiteseerGraphDataset()
# dataset = dgl.data.PubmedGraphDataset()
# dataset = dgl.data.AmazonCoBuyPhotoDataset()
# dataset = dgl.data.AmazonCoBuyComputerDataset()
g = dataset[0]

# extend_metric: "adamic_adar", "resource_alloc", "jaccard", "degree", "pagerank_degree"
# extend_metric = 'None'
# extend_metric = 'degree'
# extend_metric = 'adamic_adar'
# extend_metric = 'resource_alloc'
extend_metric = 'jaccard'
# extend_metric = 'pagerank_degree'

# model = 'sage'
# model = 'gcn'
model = 'gat'
# model = 'gatv2'

add_self_loop = True

# alpha = int(3327 - 3327*0.2)
alpha = int(2708 - 2708*0.2)
beta = 50
gamma_edges = 1500
explore = 0.4

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [3]:
# (3327 - 3327*0.2)
2708- 2708*0.2

2166.4

In [4]:
# # extend_metric = 'degree'
# extend_metric = 'pagerank_degree'

# # model = 'sage'
# model = 'gcn'
# # model = 'gat'
# # model = 'gatv2'
# add_self_loop = True

# alpha = 50000
# beta = 100
# # gamma_edges = 4000
# # explore = 0.005

In [5]:
%matplotlib inline
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import scipy.sparse as sp
import networkx as nx

import argparse
import time
import random
import itertools

from dgl.data import register_data_args

In [6]:
seed=100
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
dgl.random.seed(seed)

### Neighborhood extention functions

In [7]:
import networkx as nx
import torch
import heapq
import numpy as np
import dgl
from numpy.linalg import inv
import math
from tqdm import tqdm
import random
import numpy as np

def centrality_based(centrality_metric, graph, graph_undirected, alpha, beta, dataset):

    # these ones return a Dictionary of nodes with centrality as the value.
    if centrality_metric == 'degree':
        centrality = nx.degree_centrality(graph)
    elif centrality_metric == 'eigenvector':
        graph = nx.DiGraph(graph)
        centrality = nx.eigenvector_centrality(graph)

    important_nodes = heapq.nlargest(alpha, centrality, key=centrality.get)
    
    # TODO, should the selected nodes change for each imp_node?
    new_dataset = dataset
    for item in important_nodes:
        tgt_nodes = set(graph_undirected) - set(graph_undirected[item])
        selected_nodes = torch.tensor(np.random.choice(list(tgt_nodes), beta))
        important_node = torch.ones(len(selected_nodes), dtype=int)*torch.tensor(item)
        new_dataset = dgl.add_edges(new_dataset, selected_nodes, important_node)
        new_dataset = dgl.add_edges(new_dataset, important_node, selected_nodes)    

    return new_dataset


def non_edges_important_nodes(graph, alpha, explore):
    # nodes = set(graph)
    # selected_src_nodes = np.random.choice(list(nodes), int(0.2*len(nodes)), replace=False)
    # centrality = nx.eigenvector_centrality(graph)
    centrality = nx.degree_centrality(graph)
    selected_src_nodes = heapq.nlargest(alpha, centrality, key=centrality.get)

    edges_list = []
    for u in tqdm(selected_src_nodes):
        tgt_nodes = set(graph) - set(graph[u])
        selected_tgt_nodes = np.random.choice(list(tgt_nodes), int(explore*len(tgt_nodes)), replace=False)

        edges = list(zip(np.ones(len(selected_tgt_nodes), dtype=int)*u, selected_tgt_nodes))
        edges_list.extend(edges)

    return edges_list


def resource_allocation_index(G, dataset, gamma, alpha, explore, ebunch=None):
    if ebunch is None:
        ebunch = non_edges_important_nodes(G, alpha, explore)
    
    def predict(u, v):
        return sum(1 / G.degree(w) for w in nx.common_neighbors(G, u, v))

    edges = []
    scores = []
    for edge in tqdm(ebunch):
        u, v = edge
        try:
          pred_val = predict(u, v)
          if pred_val > 0:
              edges.append((u,v))
              scores.append(pred_val)
        except:
          continue

    scores = np.array(scores)
    sum_scores = scores.sum()
    scores /= sum_scores

    rnd_indices = np.random.choice(len(edges), gamma, p=scores, replace=True)
    selected_edges = [edges[i] for i in rnd_indices]

    new_edges = torch.tensor(selected_edges)

    new_edges_1 = new_edges[:,0]
    new_edges_2 = new_edges[:,1]

    new_dataset = dgl.add_edges(dataset, new_edges_1, new_edges_2)
    new_dataset = dgl.add_edges(new_dataset, new_edges_2, new_edges_1)

    return new_dataset


def jaccard_coefficient_index(G, dataset, gamma, alpha, explore, ebunch=None):
    if ebunch is None:
        ebunch = non_edges_important_nodes(G, alpha, explore)
    
    def predict(u, v):
        cnbors = list(nx.common_neighbors(G, u, v))
        union_size = len(set(G[u]) | set(G[v]))
        if union_size == 0:
            return 0
        else:
            return len(cnbors) / union_size

    edges = []
    scores = []
    for edge in tqdm(ebunch):
        u, v = edge
        try:
          pred_val = predict(u, v)
          if pred_val > 0:
              edges.append((u,v))
              scores.append(pred_val)
        except:
          continue

    scores = np.array(scores)
    sum_scores = scores.sum()
    scores /= sum_scores

    rnd_indices = np.random.choice(len(edges), gamma, p=scores, replace=True)
    selected_edges = [edges[i] for i in rnd_indices]

    new_edges = torch.tensor(selected_edges)

    new_edges_1 = new_edges[:,0]
    new_edges_2 = new_edges[:,1]

    new_dataset = dgl.add_edges(dataset, new_edges_1, new_edges_2)
    new_dataset = dgl.add_edges(new_dataset, new_edges_2, new_edges_1)

    return new_dataset


def adamic_adar_index(G, dataset, gamma, alpha, explore, ebunch=None):
    if ebunch is None:
        ebunch = non_edges_important_nodes(G, alpha, explore)

    def predict(u, v):
        return sum(1 / math.log(G.degree(w))
                   for w in nx.common_neighbors(G, u, v))

    edges = []
    scores = []
    for edge in tqdm(ebunch):
        u, v = edge
        try:
          pred_val = predict(u, v)
          if pred_val > 0:
              edges.append((u,v))
              scores.append(pred_val)
        except:
          continue

    scores = np.array(scores)
    sum_scores = scores.sum()
    scores /= sum_scores

    rnd_indices = np.random.choice(len(edges), gamma, p=scores, replace=True)
    selected_edges = [edges[i] for i in rnd_indices]

    new_edges = torch.tensor(selected_edges)

    new_edges_1 = new_edges[:,0]
    new_edges_2 = new_edges[:,1]

    new_dataset = dgl.add_edges(dataset, new_edges_1, new_edges_2)
    new_dataset = dgl.add_edges(new_dataset, new_edges_2, new_edges_1)

    return new_dataset


def pagerank_degree(graph, alpha, num_new_edges, dataset):
    
    centrality = nx.degree_centrality(graph)

    important_nodes = heapq.nlargest(alpha, centrality, key=centrality.get)
    
    new_dataset = dataset
    for node_id in tqdm(important_nodes):
        ppr = nx.pagerank(graph, personalization={node_id:1})
        important_tgt_nodes = torch.tensor(heapq.nlargest(num_new_edges+1, ppr, key=ppr.get))
        important_tgt_nodes = important_tgt_nodes[1:]

        node_source = torch.ones(num_new_edges, dtype=int)*torch.tensor(node_id)

        new_dataset = dgl.add_edges(new_dataset, node_source, important_tgt_nodes)
        new_dataset = dgl.add_edges(new_dataset, important_tgt_nodes, node_source)

        return new_dataset

### Prepare training and testing sets

10% for the test-set 



In [8]:
def extend_neighborhood(dataset, graph_nx, graph_undirected, alpha, beta, gamma, explore, extend_metric):
    if extend_metric == 'adamic_adar':
        print("---Adamic Adar index---")
        extended_graph = adamic_adar_index(graph_undirected, dataset=dataset, gamma=gamma, alpha=alpha, explore=explore, ebunch=None)
    elif extend_metric == 'resource_alloc':
        print("---Resouce Allocation index---")
        extended_graph = resource_allocation_index(graph_undirected, dataset=dataset, gamma=gamma, alpha=alpha, explore=explore, ebunch=None)
    elif extend_metric == 'jaccard':
        print("---Jaccard Coefficient index---")
        extended_graph = jaccard_coefficient_index(graph_undirected, dataset=dataset, gamma=gamma, alpha=alpha, explore=explore, ebunch=None)
    elif extend_metric == 'degree':
        print("---Degree index---")
        extended_graph = centrality_based(centrality_metric='degree', graph=graph_nx, graph_undirected=graph_undirected, alpha=alpha, beta=beta, dataset=dataset)
    elif extend_metric == 'pagerank_degree':
        print("---PageRank-degree index---")
        extended_graph = pagerank_degree(graph=graph_nx, alpha=alpha, num_new_edges=beta, dataset=dataset)

    return extended_graph

In [9]:
# Split edge set for training and testing
u, v = g.edges()

eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)
test_size = int(len(eids) * 0.2)
train_size = g.number_of_edges() - test_size
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

# Find all negative edges and split them for training and testing
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), g.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]

In [10]:
train_g = dgl.remove_edges(g, eids[:test_size])
graph_nx = dgl.to_networkx(train_g)
graph_undirected = nx.Graph(graph_nx)

In [11]:
train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

### GNN models

In [12]:
from dgl.nn import SAGEConv
from dgl.nn import GraphConv
from dgl.nn import GATConv
from dgl.nn import GATv2Conv

In [13]:
class GAT(nn.Module):
    def __init__(self,
                 num_layers,
                 in_dim,
                 num_hidden,
                 num_classes,
                 heads,
                 activation,
                 feat_drop,
                 attn_drop,
                 negative_slope,
                 residual):
        super(GAT, self).__init__()
        self.num_layers = num_layers
        self.gat_layers = nn.ModuleList()
        self.activation = activation
        # input projection (no residual)
        self.gat_layers.append(GATConv(
            in_dim, num_hidden, heads[0],
            feat_drop, attn_drop, negative_slope, False, self.activation))
        # hidden layers
        for l in range(1, num_layers):
            # due to multi-head, the in_dim = num_hidden * num_heads
            self.gat_layers.append(GATConv(
                num_hidden * heads[l-1], num_hidden, heads[l],
                feat_drop, attn_drop, negative_slope, residual, self.activation))
        # output projection
        self.gat_layers.append(GATConv(
            num_hidden * heads[-2], num_classes, heads[-1],
            feat_drop, attn_drop, negative_slope, residual, None))

    def forward(self, g, inputs):
        h = inputs
        for l in range(self.num_layers):
            h = self.gat_layers[l](g, h).flatten(1)
        # output projection
        logits = self.gat_layers[-1](g, h).mean(1)
        return logits

In [14]:
class GATv2(nn.Module):
    def __init__(self,
                 num_layers,
                 in_dim,
                 num_hidden,
                 num_classes,
                 heads,
                 activation,
                 feat_drop,
                 attn_drop,
                 negative_slope,
                 residual):
        super(GATv2, self).__init__()
        self.num_layers = num_layers
        self.gat_layers = nn.ModuleList()
        self.activation = activation
        # input projection (no residual)
        self.gat_layers.append(GATv2Conv(
            in_dim, num_hidden, heads[0],
            feat_drop, attn_drop, negative_slope, False, self.activation))
        # hidden layers
        for l in range(1, num_layers):
            # due to multi-head, the in_dim = num_hidden * num_heads
            self.gat_layers.append(GATv2Conv(
                num_hidden * heads[l-1], num_hidden, heads[l],
                feat_drop, attn_drop, negative_slope, residual, self.activation))
        # output projection
        self.gat_layers.append(GATv2Conv(
            num_hidden * heads[-2], num_classes, heads[-1],
            feat_drop, attn_drop, negative_slope, residual, None))

    def forward(self, g, inputs):
        h = inputs
        for l in range(self.num_layers):
            h = self.gat_layers[l](g, h).flatten(1)
        # output projection
        logits = self.gat_layers[-1](g, h).mean(1)
        return logits

In [15]:
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

class GCN(nn.Module):
  def __init__(self, in_feats, h_feats):
      super(GCN, self).__init__()
      self.conv1 = GraphConv(in_feats, h_feats)
      self.conv2 = GraphConv(h_feats, h_feats)
  
  def forward(self, g, in_feat):
      h = self.conv1(g, in_feat)
      h = F.relu(h)
      h = self.conv2(g, h)
      return h

### Model definition

In [16]:
# build a two-layer GNN model
if model == 'sage':
    print("SAGE model is used")
    model = GraphSAGE(train_g.ndata['feat'].shape[1], 16)

elif model == 'gcn':
    print("GCN model is used")
    model = GCN(train_g.ndata['feat'].shape[1], 16)

elif model == 'gat':
    print("GAT model is used")
    num_heads, num_layers, num_out_heads = 8, 2, 1
    num_hidden = n_classes = 16
    in_drop, attn_drop, negative_slope, residual = 0.6, 0.6, 0.2, False

    heads = ([num_heads] * num_layers) + [num_out_heads]
    model = GAT(num_layers, train_g.ndata['feat'].shape[1], num_hidden, n_classes, heads, F.elu, in_drop, attn_drop, negative_slope, residual)

elif model == 'gatv2':
    print("GATv2 model is used")
    num_heads, num_layers, num_out_heads = 8, 2, 1
    num_hidden = n_classes = 16
    in_drop, attn_drop, negative_slope, residual = 0.6, 0.6, 0.2, False

    heads = ([num_heads] * num_layers) + [num_out_heads]
    model = GATv2(num_layers, train_g.ndata['feat'].shape[1], num_hidden, n_classes, heads, F.elu, in_drop, attn_drop, negative_slope, residual)

else:
    raise("Not a model!")

GAT model is used


max, sum, concat, avg

In [17]:
import dgl.function as fn

class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

In [18]:
class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        # Computes a scalar score for each edge of the given graph.
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

In [19]:
class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats, h_feats//2)
        self.W2 = nn.Linear(h_feats//2, 1)

    def apply_edges(self, edges):
        # Computes a scalar score for each edge of the given graph.
        # Tested: max, min, cat, add, mean(add/2)
        h = torch.mul(edges.src['h'], edges.dst['h'])
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

In [20]:
# pred = DotPredictor()
pred = MLPPredictor(16)

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)

In [ ]:
if extend_metric != 'None':
    t1 = time.time()
    num_edge_before_extention = train_g.number_of_edges()
    print(f'Total number of edges before extension: {num_edge_before_extention}')
    
    train_g = extend_neighborhood(train_g, graph_nx, graph_undirected,  alpha, beta, gamma_edges, explore, extend_metric)
    
    t_total = time.time() - t1
    num_edge_after_extention = train_g.number_of_edges()
    print(f'Total number of edges after extension: {num_edge_after_extention}')
    print(f'Total number of added edges: {num_edge_after_extention - num_edge_before_extention}')
    print(f"*** Total construction time in seconds: {t_total:.2f} ***")

else:
    print("Neighborhood is not extended!")

Total number of edges before extension: 8445
---Jaccard Coefficient index---


 55%|█████▍    | 1279163/2341663 [00:47<00:20, 52731.43it/s]

In [ ]:
if add_self_loop == True:
    print(f"Total edges before adding self-loop {train_g.number_of_edges()}")
    train_g = train_g.remove_self_loop().add_self_loop()
    print(f"Total edges after adding self-loop {train_g.number_of_edges()}")

### Train & Evaluation

In [ ]:
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.01)

all_logits = []
for e in range(100):
    # forward
    h = model(train_g, train_g.ndata['feat'])
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

# ----------- 5. check results ------------------------ #
from sklearn.metrics import roc_auc_score
with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    print('AUC', compute_auc(pos_score, neg_score))